In [1]:
import numpy as np
import pandas as pd
import os
import json

cwd = os.getcwd()

results_path = cwd[:-28] + "Results/T5_wiki_ft_test/all_scores_range_0_137_trail_{}.npz"
results_path_sbert = cwd[:-28] + "Results/T5_wiki_ft_test/all_sbert_scores_0_137_trail_{}.npz"

results_intent_path = cwd[:-28] + "Results/T5_wiki_ft_test/all_scores_range_by_intent_0_137_trail_{}.txt"

intents = ["What drives the economy in this state?",
           "What are the available modes of transport in this state?",
           "How is the government structured in this state?",
           "What about this state's arts and culture attracts you the most?",
           "The main reasons why you would like living in this state"]

sample_size = 2

model = "T5 Wiki-Transfer"

In [2]:

total_data = None
total_data_sbert = []
total_data_intent = {}

for intent in intents:
    total_data_intent[intent] = []

# get all of the "normal" data
for i in range(sample_size):
    temp = np.load(results_path.format(i))
    sbert_temp = np.load(results_path_sbert.format(i))
    data = temp['scores']
    data_sbert = sbert_temp['scores']
    
    if total_data is None:
        # we need to take the mean of axis-one first because each "user-sample" contains a set of predicted summaries. So we first avg those sets, before average across all the data/trials
        total_data = data.mean(axis=1)
        total_data_sbert = data_sbert.mean(axis=1)
    else:
        total_data = np.concatenate((total_data, data.mean(axis=1)), axis=0)
        total_data_sbert = np.concatenate((total_data_sbert, data_sbert.mean(axis=1)), axis=0)

# get all of the "by-intent" data
for i in range(sample_size):
    res = None
    with open(results_intent_path.format(i)) as f:
        res = json.load(f)

    for intent in intents:
        total_data_intent[intent].extend(np.mean(res[intent], axis=1).tolist())

# now calculate the relavent stats
total_mean = np.mean(total_data, axis=0)
total_stddev = np.std(total_data, axis=0)

total_mean_sbert = np.mean(total_data_sbert, axis=0)
total_stddev_sbert = np.std(total_data_sbert, axis=0)

mean_intent = {}
stddev_intent = {}

for intent in intents:
    mean = np.mean(total_data_intent[intent], axis=0)
    stddev = np.std(total_data_intent[intent], axis=0)
    mean_intent[intent] = mean
    stddev_intent[intent] = stddev

metrics = ["Rouge-1", "Rouge-2", "Rouge-L"]

for i, (row_m, row_s) in enumerate(zip(total_mean, total_stddev)):
    row_str = model + " & "
    for col_m, col_s in zip(row_m, row_s):
        row_str += str(np.round(col_m, 4)) + " $\\pm$ " + str(np.round(col_s, 4)) + " & "
    
    row_str = row_str[:-3]
    row_str += " \\\\"
    print(metrics[i])
    print(row_str)
print("SBERT Cosime Sim. Score: ")
print(model + " & " + str(np.round(total_mean_sbert, 4)) + " $\\pm$ " + str(np.round(total_stddev_sbert, 4)) + " \\\\")

print("\n\n\n")

Rouge-1
T5 Wiki-Transfer & 0.1275 $\pm$ 0.035 & 0.5034 $\pm$ 0.0834 & 0.196 $\pm$ 0.0424 & 0.1225 $\pm$ 0.0265 \\
Rouge-2
T5 Wiki-Transfer & 0.03 $\pm$ 0.0171 & 0.1309 $\pm$ 0.0848 & 0.0472 $\pm$ 0.0271 & 0.0295 $\pm$ 0.0169 \\
Rouge-L
T5 Wiki-Transfer & 0.0884 $\pm$ 0.0227 & 0.3568 $\pm$ 0.0709 & 0.1366 $\pm$ 0.028 & 0.0854 $\pm$ 0.0175 \\
SBERT Cosime Sim. Score: 
T5 Wiki-Transfer & 0.4612 $\pm$ 0.0721 \\






In [3]:
metrics = ["Rouge-1", "Rouge-2", "Rouge-L"]

for intent in intents:
    print("\nResults for intent {}\n".format(intent))
    for i, (row_m, row_s) in enumerate(zip(mean_intent[intent], stddev_intent[intent])):
        row_str = "\t"+model + " & "
        for col_m, col_s in zip(row_m, row_s):
            row_str += str(np.round(col_m, 4)) + " $\\pm$ " + str(np.round(col_s, 4)) + " & "
        
        row_str = row_str[:-3]
        row_str += " \\\\"
        print("\t"+metrics[i])
        print(row_str)


Results for intent What drives the economy in this state?

	Rouge-1
	T5 Wiki-Transfer & 0.1259 $\pm$ 0.0356 & 0.4897 $\pm$ 0.0674 & 0.1945 $\pm$ 0.0438 & 0.1216 $\pm$ 0.0274 \\
	Rouge-2
	T5 Wiki-Transfer & 0.0305 $\pm$ 0.019 & 0.1242 $\pm$ 0.0651 & 0.0476 $\pm$ 0.0281 & 0.0297 $\pm$ 0.0176 \\
	Rouge-L
	T5 Wiki-Transfer & 0.0882 $\pm$ 0.0236 & 0.3494 $\pm$ 0.0565 & 0.1367 $\pm$ 0.0293 & 0.0854 $\pm$ 0.0183 \\

Results for intent What are the available modes of transport in this state?

	Rouge-1
	T5 Wiki-Transfer & 0.1306 $\pm$ 0.033 & 0.4876 $\pm$ 0.0616 & 0.1983 $\pm$ 0.0379 & 0.1239 $\pm$ 0.0237 \\
	Rouge-2
	T5 Wiki-Transfer & 0.0242 $\pm$ 0.0069 & 0.0983 $\pm$ 0.0348 & 0.0376 $\pm$ 0.0102 & 0.0235 $\pm$ 0.0064 \\
	Rouge-L
	T5 Wiki-Transfer & 0.0876 $\pm$ 0.0201 & 0.3338 $\pm$ 0.0473 & 0.1335 $\pm$ 0.0225 & 0.0835 $\pm$ 0.0141 \\

Results for intent How is the government structured in this state?

	Rouge-1
	T5 Wiki-Transfer & 0.1276 $\pm$ 0.0394 & 0.4739 $\pm$ 0.0621 & 0.192 $\pm$ 0.